In [1]:
from utils.settings import ENDPOINT_ARGS, ACCOUNT_ID
from utils.generic import display_JSON

from exabyte_api_client.endpoints.jobs import JobEndpoints
from exabyte_api_client.endpoints.materials import MaterialEndpoints
from exabyte_api_client.endpoints.workflows import WorkflowEndpoints
from exabyte_api_client.endpoints.workflows import WorkflowEndpoints
from exabyte_api_client.endpoints.raw_properties import RawPropertiesEndpoints

In [2]:
ACCOUNT_ID

'2XAeZaPHoyCokZMAB'

In [2]:
job_endpoints = JobEndpoints(*ENDPOINT_ARGS)
material_endpoints = MaterialEndpoints(*ENDPOINT_ARGS)
workflow_endpoints = WorkflowEndpoints(*ENDPOINT_ARGS)
raw_property_endpoints = RawPropertiesEndpoints(*ENDPOINT_ARGS)


## Tutorial 1: Submitting a simple job

In [23]:
default_material = material_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]
default_workflow = workflow_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]

material_id = default_material["_id"]
workflow_id = default_workflow["_id"]
owner_id = default_material["owner"]["_id"]


In [24]:
config = {
    "owner": {"_id": owner_id},
    "_material": {"_id": material_id},
    "workflow": {"_id": workflow_id},
    "name": "TEST JOB",
    "compute" : job_endpoints.get_compute(cluster = 'master-production-20160630-cluster-001.exabyte.io',ppn=1,queue='SR')

}

job = job_endpoints.create(config)
#job_endpoints.submit(job["_id"])

# This works as expected

In [19]:
default_material = material_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]
default_workflow = workflow_endpoints.list({"isDefault": True, "owner._id": ACCOUNT_ID})[0]

# Explicit pw.in files

In [22]:
workflow_body = default_workflow.copy()
with open("Si/pw.in", "r") as f:
    workflow_body["subworkflows"][0]["units"][0]["input"][0]["content"] = f.read()
workflow = workflow_endpoints.create(workflow_body)

material_id = default_material["_id"]
workflow_id = workflow["_id"]
owner_id = default_material["owner"]["_id"]

config = {
    "owner": {"_id": owner_id},
    "_material": {"_id": material_id},
    "workflow": {"_id": workflow_id},
    "name": "Explicit pw.in",
    "compute" : job_endpoints.get_compute(cluster = 'master-production-20160630-cluster-001.exabyte.io',ppn=1,queue='SR')
}

job = job_endpoints.create(config)


In [21]:
print(default_workflow["subworkflows"][0]["units"][0]["input"][0]["content"])

{% if subworkflowContext.MATERIAL_INDEX %}
{%- set input = input.perMaterial[subworkflowContext.MATERIAL_INDEX] -%}
{% endif -%}
&CONTROL
    calculation = 'scf'
    title = ''
    verbosity = 'low'
    restart_mode = '{{ input.RESTART_MODE }}'
    wf_collect = .true.
    tstress = .true.
    tprnfor = .true.
    outdir = {% raw %}'{{ JOB_WORK_DIR }}/outdir'{% endraw %}
    wfcdir = {% raw %}'{{ JOB_WORK_DIR }}/outdir'{% endraw %}
    prefix = '__prefix__'
    pseudo_dir = {% raw %}'{{ JOB_WORK_DIR }}/pseudo'{% endraw %}
/
&SYSTEM
    ibrav = {{ input.IBRAV }}
    nat = {{ input.NAT }}
    ntyp = {{ input.NTYP }}
    ecutwfc = {{ cutoffs.wavefunction }}
    ecutrho = {{ cutoffs.density }}
    occupations = 'smearing'
    degauss = 0.005
/
&ELECTRONS
    diagonalization = 'david'
    diago_david_ndim = 4
    diago_full_acc = .true.
    mixing_beta = 0.3
    startingwfc = 'atomic+random'
/
&IONS
/
&CELL
/
ATOMIC_SPECIES
{{ input.ATOMIC_SPECIES }}
ATOMIC_POSITIONS crystal
{{ input.ATOMIC_

In [20]:
workflow_body = None
with open("utils/workflow_template.json", "r") as f:
    workflow_body = json.load(f)



In [22]:
workflow = workflow_endpoints.create(workflow_body)

In [35]:
JOB_NAME='Si2_test_explicit'

material_id = default_material["_id"]
workflow_id = workflow["_id"]
owner_id = default_material["owner"]["_id"]

config = {
    "owner": {"_id": owner_id},
    "material": {"_id" : material_id},
    "workflow": {"_id": workflow_id},
    "name": JOB_NAME,
#    "compute" : job_endpoints.get_compute(cluster = 'master-production-20160630-cluster-001.exabyte.io',ppn=1,queue='SR')
}

job = job_endpoints.create(config)

In [36]:
JOB_NAME='Si2_test_explicit'

material_id = default_material["_id"]
workflow_id = workflow["_id"]
owner_id = default_material["owner"]["_id"]

config = {
    "owner": {"_id": owner_id},
    "material": {"_id" : material_id},
    "workflow": {"_id": workflow_id},
    "name": JOB_NAME,
#    "compute" : job_endpoints.get_compute(cluster = 'master-production-20160630-cluster-001.exabyte.io',ppn=1,queue='SR')
}

job = job_endpoints.create(config)
job_endpoints.submit(job["_id"])

{'_id': 'jm8hjhdmpKQb8Gsnu',
 'name': 'Si2_test_explicit',
 'owner': {'_id': '2XAeZaPHoyCokZMAB', 'slug': 'wolearyc', 'cls': 'Account'},
 'compute': {'ppn': 1,
  'nodes': 1,
  'queue': 'D',
  'timeLimit': '01:00:00',
  'notify': 'n',
  'cluster': {'fqdn': 'master-production-20160630-cluster-001.exabyte.io'},
  'timeLimitType': 'per single attempt',
  'isRestartable': True,
  'arguments': {'nimage': 1, 'npools': 1, 'nband': 1, 'ntg': 1, 'ndiag': 1}},
 '_project': {'_id': 'RjdmCYFAzMgr9wyhZ',
  'slug': 'wolearyc-default',
  'cls': 'Project'},
 'workflow': {'_id': 'vQewoaLTSYSoERYua',
  'name': 'Total Energy pw.in',
  'subworkflows': [{'_id': 'b02ea3556fa7478b29b4436d',
    'name': 'Total Energy',
    'application': {'_id': 'XXNZ9pqkerTJtfkwk',
     'name': 'espresso',
     'version': '5.4.0',
     'build': 'Default',
     'isDefault': True,
     'summary': 'Quantum Espresso',
     'updatedAt': '2023-09-14T22:52:50.217Z',
     'shortName': 'qe',
     'hasAdvancedComputeOptions': True,
   

In [ ]:
unit_flowchart_Id = default_workflow['subworkflows'][0]['units'][0]['flowchartId']
total_energy = raw_property_endpoints.get_property(job['_id'], unit_flowchart_Id, 'total_energy')

In [67]:
material_id

'CmqM5vZmhpJoHynu5'

In [13]:
job

{'_id': 'GhEcT43AtDNbXrg3z',
 'name': 'Si2_test',
 'owner': {'_id': '2XAeZaPHoyCokZMAB', 'slug': 'wolearyc', 'cls': 'Account'},
 'compute': {'ppn': 1,
  'nodes': 1,
  'queue': 'D',
  'timeLimit': '01:00:00',
  'notify': 'n',
  'cluster': {'fqdn': 'master-production-20160630-cluster-001.exabyte.io'},
  'timeLimitType': 'per single attempt',
  'isRestartable': True,
  'arguments': {'nimage': 1, 'npools': 1, 'nband': 1, 'ntg': 1, 'ndiag': 1}},
 '_project': {'_id': 'RjdmCYFAzMgr9wyhZ',
  'slug': 'wolearyc-default',
  'cls': 'Project'},
 'workflow': {'_id': '7vPbee3XokeM5D79Z',
  'name': 'Total Energy',
  'subworkflows': [{'_id': 'b02ea3556fa7478b29b4436d',
    'name': 'Total Energy',
    'application': {'_id': 'XXNZ9pqkerTJtfkwk',
     'name': 'espresso',
     'version': '5.4.0',
     'build': 'Default',
     'isDefault': True,
     'summary': 'Quantum Espresso',
     'updatedAt': '2023-09-14T22:52:50.217Z',
     'shortName': 'qe',
     'hasAdvancedComputeOptions': True,
     'updatedBy': 